## Назначение и цель проекта

Целью проекта является анализ тональности (настроения) отзывов пользователей о приложениях по тематике обучения (education) в магазине приложений AppStore.

## Выполнение проекта

### Сбор и сохранение данных для обучения и тестирования модели

Для сбора отзывов о приложениях (**100** различных) по тематике **education** будут использоваться следующие запросы к ресурсу [itunes.apple.com](https://apps.apple.com/us/genre/ios/id36):

- `https://itunes.apple.com/search?term=education&entity=software&limit=100` - поисковой запрос, который позволяет сформировать json-файл на основе параметров:
    - **entity** - вид медиа AppStore, например music или books (в данном проекте **software** - приложения);
    - **term** - тематика приложений (в данном проекте **education**);
    - **limit** - максимальное количество приложений, выдаваемых при запросе (в данном проекте **100**);


- `https://itunes.apple.com/us/rss/customerreviews/page={}/id={}/sortby=mostrecent/xml` - запрос, который позволяет сформировать xml-файл на основе параметров:
    - **page** - страница запроса (каждая страница содержит максимально 50 отзывов);
    - **id** - id приложения, отзывы которого необходимо загрузить;
    - **sortby** - сортировка для отзывов (в данном проекте **mostrecent** - самые последние).

#### Сбор данных о приложениях

Импортируем необходимые модули:

In [1]:
from json import loads              # для работы с json-файлами
from urllib.request import urlopen  # для открытия URL-адреса

Соберём данные (**ids** и **names**) о **100** различных приложениях по тематике **education**.

Для этого откроем URL-адрес `https://itunes.apple.com/search?term=education&entity=software&limit=100` с помощью функции `urlopen` модуля `urllib.request` и загрузим данные json-формата в переменную `apps_dict` c помощью функции `loads` модуля `json`:

In [2]:
source = 'https://itunes.apple.com/search?term=education&entity=software&limit=100'
with urlopen(source) as url:
    apps_dict = loads(url.read().decode())

В переменную `apps_dict` загрузится словарь вида:

```
{'resultCount': 5,
 'results': [{...}, ..., {...}]}
```

По ключу `results` станет доступен список из 100 словарей, каждый из которых содержит информацию о конкретном приложении. Информация об **id** и **name** для каждого приложения доступна по ключам `trackId` и `trackName` соответственно:

In [3]:
print(apps_dict['results'][0]['trackId'])    # информация об id первого приложения списка
print(apps_dict['results'][0]['trackName'])  # информация о title первого приложения списка

570060128
Duolingo - Language Lessons


Получим кортеж кортежей **ids** и **names** для каждого приложения:

In [4]:
apps = tuple((el['trackId'], el['trackName']) for el in apps_dict['results'])
print(apps[0])

(570060128, 'Duolingo - Language Lessons')


#### Создание датасетов для обучения модели и для проверки работы модели

Предварительно установим (если не установлены) необходимые модули в виртуальное окружение проекта, используя следующую команду:
```
pip3 install numpy pandas lxml langdetect spacy sklearn matplotlib
```

Импортируем необходимые модули:

In [5]:
import numpy as np
import pandas as pd

Сформируем на основе полученной информации два data frames, содержащие отзывы о приложениях:
- для обучения модели будет использоваться 2 страница (page=2) отзывов;
- для проверки модели будет использоваться 1 страница (page=1) отзывов.

In [23]:
df = pd.DataFrame()        # инициализация DataFrame для обучения модели
df_check = pd.DataFrame()  # инициализация DataFrame для проверки модели

# цикл по 100 приложениям
for app_id, app_name in apps:
    
    # создание запросов с app_id
    url = f'https://itunes.apple.com/us/rss/customerreviews/page=2/id={app_id}/sortby=mostrecent/xml'
    url_check = f'https://itunes.apple.com/us/rss/customerreviews/page=1/id={app_id}/sortby=mostrecent/xml'
    
    # считывание xml-файла в Pandas DataFrame
    new_df = pd.read_xml(url, parser='lxml')  # lxml - требуемый парсер
    new_df_check = pd.read_xml(url_check, parser='lxml')
    
    # сохранение только нужных столбцов updated, content, rating
    cols = ['updated', 'content', 'rating']
    new_df = new_df.reindex(columns=cols)
    new_df_check = new_df_check.reindex(columns=cols)
    
    # и удалениe строк, в которых содержится общая информация 
    # о документе xml, а не сами обзоры (rating = NaN)
    new_df = new_df[new_df['rating'].notna()]
    new_df_check = new_df_check[new_df_check['rating'].notna()]
    
    # добавление столбца с именем приложения app_name
    new_df['app_name'] = app_name
    new_df_check['app_name'] = app_name
    
    # сохранение загруженных данных в общий DataFrame
    df = pd.concat([df, new_df], ignore_index=True)
    df_check = pd.concat([df_check, new_df_check], ignore_index=True)

In [26]:
df.head()

,updated,content,rating,app_name
0,2022-02-28T19:05:27-07:00,"<table border=""0"" width=""100%"">\n <tr>\n ...",1.0,Duolingo - Language Lessons
1,2022-02-28T18:34:53-07:00,"<table border=""0"" width=""100%"">\n <tr>\n ...",4.0,Duolingo - Language Lessons
2,2022-02-28T18:09:45-07:00,"<table border=""0"" width=""100%"">\n <tr>\n ...",4.0,Duolingo - Language Lessons
3,2022-02-28T17:59:14-07:00,"<table border=""0"" width=""100%"">\n <tr>\n ...",3.0,Duolingo - Language Lessons
4,2022-02-28T16:33:56-07:00,"<table border=""0"" width=""100%"">\n <tr>\n ...",2.0,Duolingo - Language Lessons


In [ ]:
df['timestamp'] = pd.to_datetime(df['timestamp'])
reg = '^.+(\\n.*)+\"><br\/>|<\/font><br\/>(\\n.*)+$'
df['content'].replace(reg, '', regex=True, inplace=True)
df['content'].head(150)